# 朴素贝叶斯

## 使用朴素贝叶斯进行文档分类

## 案例1 文本分类

    以社区在线留言板为背景，实现判断言论是否含有侮辱性文字。


> 1 从文中构建词向量

In [18]:
def loadDataSet():
    """
    postingList:实验样本
    classVec:
        label=1 有侮辱性文字
        label=0 正常言论
    """
    postingList = [
        ['my', 'dog', 'has', 'flea', 'problems', 'help', 'please'],
        ['maybe', 'not', 'take', 'him', 'to', 'dog', 'park', 'stupid'],
        ['my', 'dalmation', 'is', 'so', 'cute', 'I', 'love', 'him'],
        ['stop', 'posting', 'stupid', 'worthless', 'garbage'],
        ['mr', 'licks', 'ate', 'my', 'steak', 'how', 'to', 'stop', 'him'],
        ['quit', 'buying', 'worthless', 'dog', 'food', 'stupid']
    ]
    # 'my dog has flea problems help please'.split()
    # 'maybe not take him to dog park stupid'.split()
    # 'my dalmation is so cute I love him'.split()
    # 'stop posting stupid worthless garbage'.split()
    # 'mr licks ate my steak how to stop him'.split()
    # 'quit buying worthless dog food stupid'.split()
    classVec = [0, 1, 0, 1, 0, 1]
    return postingList, classVec
    
    
def createVocabList(dataSet):
    """
    获取dataSet中的所有单词集合
    """
    vocabSet = set([])
    for document in dataSet:
        vocabSet = vocabSet | set(document)
    return list(vocabSet)


def setOfWords2Vec(vocabList, inputSet):
    """
    将文档转成文档向量
    """
    returnVec = [0] * len(vocabList)
    for word in inputSet:
        if word in vocabList:
            returnVec[vocabList.index(word)] = 1
        else:
            print('the word: {} is not in my Vocabulary!'.format(word))
    return returnVec

> 从词向量计算概率

In [19]:
import numpy as np

def trainNB0(trainMatrix, trainCategory):
    numTrainDocs = len(trainMatrix)
    numWords = len(trainMatrix[0])
    pAbusive = sum(trainCategory) / float(numTrainDocs)
    p0Num, p1Num = np.ones(numWords), np.ones(numWords)
    p0Denom, p1Denom = 2.0, 2.0
    for i in range(numTrainDocs):
        if trainCategory[i] == 1:
            p1Num += trainMatrix[i]
            p1Denom += sum(trainMatrix[i])
        else:
            p0Num += trainMatrix[i]
            p0Denom += sum(trainMatrix[i])
    p1Vect = np.log(p1Num / p1Denom)
    p0Vect = np.log(p0Num / p0Denom)
    return p0Vect, p1Vect, pAbusive


> 朴素贝叶斯分类函数

In [23]:
def classifyNB(vec2Classify, p0Vec, p1Vec, pClass1):
    p1 = sum(vec2Classify * p1Vec) + np.log(pClass1)
    p0 = sum(vec2Classify * p0Vec) + np.log(1.0 - pClass1)
    
    if p1 > p0:
        return 1
    else:
        return 0

> main

In [24]:
def testingNB():
    # get train data
    listOPosts, listClasses = loadDataSet()
    # get word vector
    myVocabList = createVocabList(listOPosts)
    # word to vector
    trainMat = []
    for postinDoc in listOPosts:
        trainMat.append(setOfWords2Vec(myVocabList, postinDoc))
    
    # 
    p0V, p1V, pAb = trainNB0(np.array(trainMat), np.array(listClasses))
    
    testEntry = 'love my dalmation'.split()
    thisDoc = np.array(setOfWords2Vec(myVocabList, testEntry))
    print('classified as:', classifyNB(thisDoc, p0V, p1V, pAb))
    
    testEntry = 'stupid garbage'.split()
    thisDoc = np.array(setOfWords2Vec(myVocabList, testEntry))
    print('classified as:', classifyNB(thisDoc, p0V, p1V, pAb))

In [25]:
testingNB()

classified as: 0
classified as: 1
